In [1]:
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd
import geocoder
from geopy.geocoders import Nominatim 
from datetime import datetime 

In [2]:
direccion_usuario= input("Please, Enter your location: ")

Please, Enter your location: calle juan ramon jimenez, 15, alcobendas


In [3]:
dia_hoy=datetime.today().strftime('%d-%m')
df_new= pd.read_excel("df_diario 30-04-2022.xlsx")
#df_new= pd.read_excel("prueba_colores.xlsx")

In [5]:
df_new

,Provincia,Municipio,Localidad,Código postal,Dirección,Longitud,Latitud,Precio gasolina 95 E5,Precio gasolina 95 E5 Premium,Precio gasolina 98 E5,Precio gasóleo A,Precio gasóleo Premium,Rótulo,Horario,Fecha de extracción
0,ALBACETE,Abengibre,ABENGIBRE,2250,"AVENIDA CASTILLA LA MANCHA, 26","-1,539167","39,211417","1,779",No disponible,No disponible,"1,839",No disponible,Nº 10.935,L-D: 07:00-22:00,30-04-2022
1,ALBACETE,Alatoz,ALATOZ,2152,"CR CM-332, 46,4","-1,346083","39,100389","1,849",No disponible,"2,000","1,890","2,000",REPSOL,L-D: 7:00-23:00,30-04-2022
2,ALBACETE,Albacete,ALBACETE,2001,"AVENIDA 1º DE MAYO, S/N","-1,868500","38,985667","1,839",No disponible,"1,939","1,889","1,919",CARREFOUR,L-S: 08:00-22:00; D: 09:00-21:00,30-04-2022
3,ALBACETE,Albacete,ALBACETE,2001,"CALLE FEDERICO GARCIA LORCA, 5","-1,846361","38,997722",No disponible,No disponible,No disponible,"1,929",No disponible,CEPSA,L-S: 05:00-23:00,30-04-2022
4,ALBACETE,Albacete,ALBACETE,2001,CALLE PRINCIPE DE ASTURIAS (POLÍGONO DE ROMICA...,"-1,832000","39,054694","1,889",No disponible,"2,031","1,959","2,049",BP ROMICA,L-D: 06:00-21:30,30-04-2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11656,ZARAGOZA,Zaragoza,ZARAGOZA,50197,"RONDA FERIA DE MUESTRAS, DE LA, 20","-0,986389","41,637944",No disponible,No disponible,No disponible,"1,929",No disponible,IDS,L-D: 24H,30-04-2022
11657,ZARAGOZA,Zaragoza,ZARAGOZA,50197,"POLIGONO CALLE TURIASO, 31","-0,974194","41,645722",No disponible,No disponible,No disponible,"1,889",No disponible,NOIL,L-D: 24H,30-04-2022
11658,ZARAGOZA,Zuera,ZUERA,50800,"CARRETERA N-330A KM. 24,000","-0,794000","41,857000","1,899",No disponible,"2,029","1,919","1,989",REPSOL,L-D: 06:00-22:00,30-04-2022
11659,ZARAGOZA,Zuera,ZUERA,50800,"CALLE AUSTRIA, 11,17","-0,782833","41,883583","1,817",No disponible,No disponible,"1,828","1,868",COOPERATIVA SAN LICER,L-D: 24H,30-04-2022


In [6]:
df_new['Latitud'] = df_new['Latitud'].apply(lambda x: x.replace(',','.'))
df_new['Longitud'] = df_new['Longitud'].apply(lambda x: x.replace(',','.'))

In [8]:
geolocator = Nominatim(user_agent="My-App")
location = geolocator.geocode(direccion_usuario) 
#latitud_partida=location.latitude
#longitud_partida=location.longitude
#latitud_partida
#longitud_partida


In [9]:
df_new['Latitud_partida'] = location.latitude
df_new['Longitud_partida'] = location.longitude

df_new['Latitud']=df_new['Latitud'].astype('float64')
df_new['Longitud']=df_new['Longitud'].astype('float64')

df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11661 entries, 0 to 11660
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Provincia                      11661 non-null  object 
 1   Municipio                      11661 non-null  object 
 2   Localidad                      11661 non-null  object 
 3   Código postal                  11661 non-null  int64  
 4   Dirección                      11661 non-null  object 
 5   Longitud                       11661 non-null  float64
 6   Latitud                        11661 non-null  float64
 7   Precio gasolina 95 E5          11661 non-null  object 
 8   Precio gasolina 95 E5 Premium  11661 non-null  object 
 9   Precio gasolina 98 E5          11661 non-null  object 
 10  Precio gasóleo A               11661 non-null  object 
 11  Precio gasóleo Premium         11661 non-null  object 
 12  Rótulo                         11661 non-null 

In [10]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(mercator_start, mercator_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    return mercator_start.distance(mercator_finish)

In [11]:
#si quiero hacer un DF más pequeño en el que aplicar to mercator, hay que crearlo aquí

In [12]:
df_new["mercator_start"] = df_new.apply(lambda x: to_mercator(x['Latitud_partida'],x['Longitud_partida']),axis=1)
df_new["mercator_finish"] = df_new.apply(lambda x: to_mercator(x['Latitud'],x['Longitud']),axis=1)




In [13]:

#calcular distancia
df_new["Distance"] = df_new.apply(lambda x: distance_meters(x['mercator_start'],x['mercator_finish']),axis=1)


In [14]:
df_new.to_excel("prueba_visualizacion.xlsx", index=False)

## Funciones para el pipeline

In [34]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(mercator_start, mercator_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    return mercator_start.distance(mercator_finish)

In [35]:
direccion_usuario= input("Please, Enter your location: ")

Please, Enter your location: calle juan ramón jimenez, 15, alcobendas


In [ ]:

def mercator_gas():
    geolocator = Nominatim(user_agent="My-App")
    location = geolocator.geocode(direccion_usuario) 
    df_new['Latitud'] = df_new['Latitud'].apply(lambda x: x.replace(',','.'))
    df_new['Longitud'] = df_new['Longitud'].apply(lambda x: x.replace(',','.'))
    df_new['Latitud_partida'] = location.latitude
    df_new['Longitud_partida'] = location.longitude
    df_new['Latitud']=df_new['Latitud'].astype('float64')
    df_new['Longitud']=df_new['Longitud'].astype('float64')
    df_new["mercator_start"] = df_new.apply(lambda x: to_mercator(x['Latitud_partida'],x['Longitud_partida']),axis=1)
    df_new["mercator_finish"] = df_new.apply(lambda x: to_mercator(x['Latitud'],x['Longitud']),axis=1)
    return df_new


In [36]:




def mercator_gas():
    geolocator = Nominatim(user_agent="My-App")
    location = geolocator.geocode(direccion_usuario) 
    df_new['Latitud'] = df_new['Latitud'].apply(lambda x: x.replace(',','.'))
    df_new['Longitud'] = df_new['Longitud'].apply(lambda x: x.replace(',','.'))
    df_new['Latitud_partida'] = location.latitude
    df_new['Longitud_partida'] = location.longitude
    df_new['Latitud']=df_new['Latitud'].astype('float64')
    df_new['Longitud']=df_new['Longitud'].astype('float64')
    df_new["mercator_start"] = df_new.apply(lambda x: to_mercator(x['Latitud_partida'],x['Longitud_partida']),axis=1)
    df_new["mercator_finish"] = df_new.apply(lambda x: to_mercator(x['Latitud'],x['Longitud']),axis=1)
    return df_new

In [37]:
df_new = mercator_gas()

In [38]:
df_new #aqui tengo el df sin la columna de distancia

,Provincia,Municipio,Localidad,Código postal,Dirección,Longitud,Latitud,Precio gasolina 95 E5,Precio gasolina 95 E5 Premium,Precio gasolina 98 E5,Precio gasóleo A,Precio gasóleo Premium,Rótulo,Horario,Fecha de extracción,Latitud_partida,Longitud_partida,mercator_start,mercator_finish
0,ALBACETE,Abengibre,ABENGIBRE,2250,"AVENIDA CASTILLA LA MANCHA, 26",-1.539167,39.211417,"1,779",No disponible,No disponible,"1,839",No disponible,Nº 10.935,L-D: 07:00-22:00,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4364994.97372271 -171359.8982185098)
1,ALBACETE,Alatoz,ALATOZ,2152,"CR CM-332, 46,4",-1.346083,39.100389,"1,849",No disponible,"2,000","1,890","2,000",REPSOL,L-D: 7:00-23:00,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4352635.393298916 -149859.0605023663)
2,ALBACETE,Albacete,ALBACETE,2001,"AVENIDA 1º DE MAYO, S/N",-1.868500,38.985667,"1,839",No disponible,"1,939","1,889","1,919",CARREFOUR,L-S: 08:00-22:00; D: 09:00-21:00,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4339864.598676129 -208037.3467282484)
3,ALBACETE,Albacete,ALBACETE,2001,"CALLE FEDERICO GARCIA LORCA, 5",-1.846361,38.997722,No disponible,No disponible,No disponible,"1,929",No disponible,CEPSA,L-S: 05:00-23:00,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4341206.555137643 -205571.5489110633)
4,ALBACETE,Albacete,ALBACETE,2001,CALLE PRINCIPE DE ASTURIAS (POLÍGONO DE ROMICA...,-1.832000,39.054694,"1,889",No disponible,"2,031","1,959","2,049",BP ROMICA,L-D: 06:00-21:30,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4347548.649167117 -203972.0657211719)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11656,ZARAGOZA,Zaragoza,ZARAGOZA,50197,"RONDA FERIA DE MUESTRAS, DE LA, 20",-0.986389,41.637944,No disponible,No disponible,No disponible,"1,929",No disponible,IDS,L-D: 24H,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4635114.72375884 -109809.7456019723)
11657,ZARAGOZA,Zaragoza,ZARAGOZA,50197,"POLIGONO CALLE TURIASO, 31",-0.974194,41.645722,No disponible,No disponible,No disponible,"1,889",No disponible,NOIL,L-D: 24H,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4635980.56675823 -108452.0056893557)
11658,ZARAGOZA,Zuera,ZUERA,50800,"CARRETERA N-330A KM. 24,000",-0.794000,41.857000,"1,899",No disponible,"2,029","1,919","1,989",REPSOL,L-D: 06:00-22:00,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4659499.926134052 -88390.50484689715)
11659,ZARAGOZA,Zuera,ZUERA,50800,"CALLE AUSTRIA, 11,17",-0.782833,41.883583,"1,817",No disponible,No disponible,"1,828","1,868",COOPERATIVA SAN LICER,L-D: 24H,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4662459.13215781 -87147.28239078527)


In [39]:
def apply_distance():
    df_new["Distance"] = df_new.apply(lambda x: distance_meters(x['mercator_start'],x['mercator_finish']),axis=1)
    return df_new




In [40]:
df_new=apply_distance()

In [44]:
df_new=df_new.sort_values(by=['Distance'])
df_head=df_new.head(10)
df_head

,Provincia,Municipio,Localidad,Código postal,Dirección,Longitud,Latitud,Precio gasolina 95 E5,Precio gasolina 95 E5 Premium,Precio gasolina 98 E5,Precio gasóleo A,Precio gasóleo Premium,Rótulo,Horario,Fecha de extracción,Latitud_partida,Longitud_partida,mercator_start,mercator_finish,Distance
6387,MADRID,Alcobendas,ALCOBENDAS,28100,"AVENIDA VALDELAPARRA (DE), 5",-3.662056,40.548306,"1,924",No disponible,"2,074","1,964","2,034",GALP,L-D: 24H,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4513816.776449839 -407936.0483483151),447.131891
6391,MADRID,Alcobendas,ALCOBENDAS,28100,"CALLE MARQUES DE LA VALDAVIA (DEL), 138",-3.653944,40.546944,"1,909",No disponible,"2,051","1,969","2,059",BP VALDAVIA,L-D: 24H,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4513665.159303379 -407031.1811113249),516.618744
6385,MADRID,Alcobendas,ALCOBENDAS,28100,"AVENIDA MARQUES DE LA VALDAVIA, 132",-3.651861,40.546556,"1,799",No disponible,No disponible,"1,799",No disponible,PETROPRIX,L-D: 24H,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4513621.967340952 -406798.8305516534),745.354730
6388,MADRID,Alcobendas,ALCOBENDAS,28100,"CALLE VALGRANDE, 30",-3.654139,40.539583,"1,799",No disponible,No disponible,"1,799","1,819",BALLENOIL,L-D: 24H,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4512845.73653165 -407052.9326315577),901.605721
6403,MADRID,Alcobendas,ALCOBENDAS,28108,"CALLE VALGRANDE, 11",-3.649750,40.536389,"1,799",No disponible,No disponible,"1,799",No disponible,PETROPRIX,L-D: 24H,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4512490.182078056 -406563.3572507771),1482.389399
6399,MADRID,Alcobendas,ALCOBENDAS,28108,"CALLE FRANCISCO GERVAS, 13",-3.652250,40.533722,"1,899",No disponible,"2,049","1,939","2,039",REPSOL,L-V: 06:00-22:00; S: 07:00-14:00,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4512193.292996109 -406842.2219498365),1573.731913
6395,MADRID,Alcobendas,ALCOBENDAS,28100,"AVENIDA MONTE DE VALDELATAS, 5",-3.659667,40.528722,"1,909",No disponible,"2,051","1,969","2,059",BP EL SAPO-ALCOBENDAS,L-D: 06:00-13:59 y 14:00-22:00,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4511636.695542145 -407669.5623112876),1969.246769
6397,MADRID,Alcobendas,ALCOBENDAS,28100,"AVENIDA VALDELAPARRA, 3",-3.643194,40.537250,"1,859",No disponible,"1,999","1,899","1,979",SHELL,L-S: 06:00-22:00; D: 09:00-17:00,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4512586.02815963 -405832.0661490023),1990.599349
6396,MADRID,Alcobendas,ALCOBENDAS,28100,"CARRETERA M-603 KM. 14,900",-3.651583,40.528806,"1,859",No disponible,"1,999","1,899","1,979",SHELL,L-D: 06:00-22:00,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4511646.046379371 -406767.8207721841),2104.234713
6398,MADRID,Alcobendas,ALCOBENDAS,28100,"CARRETERA A-1 KM. 14,500",-3.649417,40.528194,"1,899",No disponible,"2,039","1,939","1,969",CARREFOUR,L-D: 24H,30-04-2022,40.546376,-3.658541,POINT (4513601.929832609 -407543.9158972811),POINT (4511577.918851005 -406526.2125314499),2265.466971


In [ ]:
df_new.to_excel('price_today.xlsx', index=False)

In [3]:
df_new

,Provincia,Municipio,Localidad,Código postal,Dirección,Longitud,Latitud,Precio gasolina 95 E5,Precio gasolina 95 E5 Premium,Precio gasolina 98 E5,Precio gasóleo A,Precio gasóleo Premium,Rótulo,Horario,Fecha de extracción
0,ALBACETE,Abengibre,ABENGIBRE,2250,"AVENIDA CASTILLA LA MANCHA, 26","-1,539167","39,211417","1,779",No disponible,No disponible,"1,839",No disponible,Nº 10.935,L-D: 07:00-22:00,30-04-2022
1,ALBACETE,Alatoz,ALATOZ,2152,"CR CM-332, 46,4","-1,346083","39,100389","1,849",No disponible,"2,000","1,890","2,000",REPSOL,L-D: 7:00-23:00,30-04-2022
2,ALBACETE,Albacete,ALBACETE,2001,"AVENIDA 1º DE MAYO, S/N","-1,868500","38,985667","1,839",No disponible,"1,939","1,889","1,919",CARREFOUR,L-S: 08:00-22:00; D: 09:00-21:00,30-04-2022
3,ALBACETE,Albacete,ALBACETE,2001,"CALLE FEDERICO GARCIA LORCA, 5","-1,846361","38,997722",No disponible,No disponible,No disponible,"1,929",No disponible,CEPSA,L-S: 05:00-23:00,30-04-2022
4,ALBACETE,Albacete,ALBACETE,2001,CALLE PRINCIPE DE ASTURIAS (POLÍGONO DE ROMICA...,"-1,832000","39,054694","1,889",No disponible,"2,031","1,959","2,049",BP ROMICA,L-D: 06:00-21:30,30-04-2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11656,ZARAGOZA,Zaragoza,ZARAGOZA,50197,"RONDA FERIA DE MUESTRAS, DE LA, 20","-0,986389","41,637944",No disponible,No disponible,No disponible,"1,929",No disponible,IDS,L-D: 24H,30-04-2022
11657,ZARAGOZA,Zaragoza,ZARAGOZA,50197,"POLIGONO CALLE TURIASO, 31","-0,974194","41,645722",No disponible,No disponible,No disponible,"1,889",No disponible,NOIL,L-D: 24H,30-04-2022
11658,ZARAGOZA,Zuera,ZUERA,50800,"CARRETERA N-330A KM. 24,000","-0,794000","41,857000","1,899",No disponible,"2,029","1,919","1,989",REPSOL,L-D: 06:00-22:00,30-04-2022
11659,ZARAGOZA,Zuera,ZUERA,50800,"CALLE AUSTRIA, 11,17","-0,782833","41,883583","1,817",No disponible,No disponible,"1,828","1,868",COOPERATIVA SAN LICER,L-D: 24H,30-04-2022


In [13]:

def mercator_gas(df_new):
    geolocator = Nominatim(user_agent="My-App")
    location = geolocator.geocode(input("Please, Enter your location: "))
    df_new['Latitud'] = df_new['Latitud'].apply(lambda x: str(x).replace(',','.'))
    df_new['Longitud'] = df_new['Longitud'].apply(lambda x: str(x).replace(',','.'))
    df_new['Latitud_partida'] = location.latitude
    df_new['Longitud_partida'] = location.longitude
    df_new['Latitud']=df_new['Latitud'].astype('float64')
    df_new['Longitud']=df_new['Longitud'].astype('float64')
    df_new["mercator_start"] = df_new.apply(lambda x: to_mercator(x['Latitud_partida'],x['Longitud_partida']),axis=1)
    df_new["mercator_finish"] = df_new.apply(lambda x: to_mercator(x['Latitud'],x['Longitud']),axis=1)
    df_new["Distance"] = df_new.apply(lambda x: distance_meters(x['mercator_start'],x['mercator_finish']),axis=1)
    df_new.sort_values(by=['Distance'])
    df_new=df_new.head(10)
    return df_new

In [14]:
df_new = mercator_gas(df_new)

Please, Enter your location: calle juan ramón jimenez, 15, alcobendas
Index(['Provincia', 'Municipio', 'Localidad', 'Código postal', 'Dirección',
       'Longitud', 'Latitud', 'Precio gasolina 95 E5',
       'Precio gasolina 95 E5 Premium', 'Precio gasolina 98 E5',
       'Precio gasóleo A', 'Precio gasóleo Premium', 'Rótulo', 'Horario',
       'Fecha de extracción', 'Latitud_partida', 'Longitud_partida',
       'mercator_start', 'mercator_finish', 'Distance'],
      dtype='object')
0    ABENGIBRE
1       ALATOZ
2     ALBACETE
3     ALBACETE
4     ALBACETE
5     ALBACETE
6     ALBACETE
7     ALBACETE
8     ALBACETE
9     ALBACETE
Name: Localidad, dtype: object
